In [1]:
import pandas as pd
import re
pd.options.mode.chained_assignment = None
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
overview = pd.DataFrame()
countries = ['Albania', 'Argentina', 'Austria', 'Bangladesh', 'Belgium', 'Bolivia', 'Brazil', 
             'Bulgaria', 'Canada', 'Chile', 'China', 'Colombia', 'Costa Rica', 'Croatia', 'Czech Republic', 
             'Denmark', 'Ecuador', 'Egypt', 'El Salvador', 'Estonia', 'Finland', 'France', 'Georgia', 'Germany', 
             'Greece', 'Guatemala', 'Hong Kong', 'Hungary',  'Indonesia', 'Iran', 'Ireland', 'Israel', 
             'Italy', 'Jamaica', 'Japan', 'Kazakhstan', 'Kuwait', 'Luxembourg','Malaysia', 'Malta', 'Mexico', 
             'Morocco', 'Namibia', 'Netherlands', 'New Zealand', 'Nigeria', 'Norway', 'Pakistan', 'Panama', 'Peru', 
             'Philippines', 'Poland', 
             'Portugal', 'Qatar', 'Romania', 'Russia', 'Serbia', 'Singapore', 'Slovakia', 'Slovenia', 'South Africa', 
             'South Korea', 'Spain', 'Suriname', 'Sweden', 'Switzerland', 'Taiwan', 'Thailand', 'Trinidad and Tobago', 
             'Turkey','Uruguay', 'Venezuela', 'Vietnam', 'Zambia', 'Zimbabwe','Australia','India','United Kingdom',
             'United States']

overview['Country'] = countries
overview['positive'] = ""
overview['negative'] = ""
overview['neutral'] = ""

In [3]:
camp = 'appreciate'

In [4]:
l = [1,2,3]
for c in countries:
    country = c
    for t in l:
        if t == 1:
            sentiment ='positive'
        elif t == 2:
            sentiment ='negative'
        else:
            sentiment ='neutral'

        fileName = camp + '_campaign_' + country

        #path to campaign tweets
        path = ('campaign_category_results/campaign_category_results/'+camp +'_campaign/' + fileName + ".csv")

        pathSaveVisual = ('ta_result2/'+camp+'/'+country+'/')


        data = pd.read_csv(path)
        data_text = data[['Cleaned_Tweet','Tweet_lemmatized','sentiment']]

        documents = data_text

        df1 = documents[documents['sentiment'] >= 4].copy(deep=True)
        df1['ta_tweet'] = df1['Tweet_lemmatized'].apply(lambda x: x[1:-1])
        df1['ta_tweet'] = df1['ta_tweet'].str.replace(r"[\"\',]", '',regex=True)

        df2 = documents[documents['sentiment'] <= 2].copy(deep=True)
        df2['ta_tweet'] = df2['Tweet_lemmatized'].apply(lambda x: x[1:-1])
        df2['ta_tweet'] = df2['ta_tweet'].str.replace(r"[\"\',]", '',regex=True)

        df3 = documents[documents['sentiment'] == 3].copy(deep=True)
        df3['ta_tweet'] = df3['Tweet_lemmatized'].apply(lambda x: x[1:-1])
        df3['ta_tweet'] = df3['ta_tweet'].str.replace(r"[\"\',]", '',regex=True)


        #Adjust df1,df2,df3 as well as sentiment according 1 being positive , 2 being negative , 3 being neutral

        if t == 1:
            tweets = df1['ta_tweet'].tolist()
        elif t == 2:
            tweets = df2['ta_tweet'].tolist()
        else:
            tweets = df3['ta_tweet'].tolist()

        vectorizer_model = CountVectorizer(ngram_range=(2, 3))
        topic_model = BERTopic(verbose=True,vectorizer_model=vectorizer_model)
        
        try:
            topics, probs = topic_model.fit_transform(tweets)
        except:
              print("An exception occurred")
                
#         topic_model.update_topics(tweets, topics, n_gram_range=(2, 3))

        try:
            if(len(topic_model.generate_topic_labels(separator=','))>20):
                new_topics, new_probs = topic_model.reduce_topics(tweets, topics, nr_topics=20)
                
            topic_model.generate_topic_labels(separator=',')
                        
            topic_keys = topic_model.get_topic_info()
            topic_keys.rename(columns ={'Name': 'TopicLabel' }, inplace=True)
            topic_keys["WordScore"] = ""
            topic_keys["RepresentativeTweets"] = ""
            topic_keys["Sentiment"] = sentiment 
            
            for i in range(0,len(topic_model.get_topic_info())):
                topic_keys.at[i, 'WordScore'] = topic_model.get_topic(i-1)
                
            for i in range(1,len(topic_model.get_topic_info())):
                try:
                    topic_keys.at[i, 'RepresentativeTweets'] = str(topic_model.get_representative_docs(i-1)[0:3])
                except:
                    topic_keys.at[i, 'RepresentativeTweets'] = "NO VALID TWEETS"
                    
            topic_keys.to_csv(pathSaveVisual+"topicinfo_"+camp+"_"+country+"_"+sentiment+".csv")

            if( len(topic_model.generate_topic_labels(separator=','))>=5):
                fig3 = topic_model.visualize_topics()
                fig3.write_html(pathSaveVisual+"visualize_topics_"+camp+"_"+sentiment+"_bert.html")
                
            print(country+" "+ sentiment+" topics generated")   
            overview.loc[overview.Country==c,sentiment]= 1
        except Exception as e:
            print(e)
            print(country+" "+ sentiment+" Dataset not big enough")
            overview.loc[overview.Country==c,sentiment]= 0
            

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

2022-08-05 20:53:32,474 - BERTopic - Transformed documents to Embeddings
2022-08-05 20:53:37,970 - BERTopic - Reduced dimensionality
2022-08-05 20:53:37,987 - BERTopic - Clustered reduced embeddings


Albania positive topics generated


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2022-08-05 20:53:42,531 - BERTopic - Transformed documents to Embeddings
2022-08-05 20:53:44,296 - BERTopic - Reduced dimensionality
2022-08-05 20:53:44,306 - BERTopic - Clustered reduced embeddings


...

2022-08-05 22:49:24,062 - BERTopic - Transformed documents to Embeddings
2022-08-05 22:49:51,712 - BERTopic - Reduced dimensionality
2022-08-05 22:49:55,171 - BERTopic - Clustered reduced embeddings
2022-08-05 22:50:34,530 - BERTopic - Reduced number of topics from 804 to 21


United Kingdom negative topics generated


Batches:   0%|          | 0/441 [00:00<?, ?it/s]

2022-08-05 22:52:36,266 - BERTopic - Transformed documents to Embeddings
2022-08-05 22:52:43,406 - BERTopic - Reduced dimensionality
2022-08-05 22:52:44,143 - BERTopic - Clustered reduced embeddings
2022-08-05 22:52:55,523 - BERTopic - Reduced number of topics from 279 to 21


United Kingdom neutral topics generated


In [5]:
overview
overview.to_csv('ta_result2/overall_'+camp+'_topics.csv')

In [12]:
# import os

# countries = ['Albania', 'Argentina', 'Australia', 'Austria', 'Bangladesh', 'Belgium', 'Bolivia', 'Brazil', 
#              'Bulgaria', 'Canada', 'Chile', 'China', 'Colombia', 'Costa Rica', 'Croatia', 'Czech Republic', 
#              'Denmark', 'Ecuador', 'Egypt', 'El Salvador', 'Estonia', 'Finland', 'France', 'Georgia', 'Germany', 
#              'Greece', 'Guatemala', 'Hong Kong', 'Hungary', 'India', 'Indonesia', 'Iran', 'Ireland', 'Israel', 
#              'Italy', 'Jamaica', 'Japan', 'Kazakhstan', 'Kuwait', 'Luxembourg','Malaysia', 'Malta', 'Mexico', 
#              'Morocco', 'Namibia', 'Netherlands', 'New Zealand', 'Nigeria', 'Norway', 'Pakistan', 'Panama', 'Peru', 
#              'Philippines', 'Poland', 
#              'Portugal', 'Qatar', 'Romania', 'Russia', 'Serbia', 'Singapore', 'Slovakia', 'Slovenia', 'South Africa', 
#              'South Korea', 'Spain', 'Suriname', 'Sweden', 'Switzerland', 'Taiwan', 'Thailand', 'Trinidad and Tobago', 
#              'Turkey', 'United Kingdom', 'United States', 'Uruguay', 'Venezuela', 'Vietnam', 'Zambia', 'Zimbabwe']

# for c in countries:
#     if not os.path.exists("ta_result2/gifts/" + c):
#         os.makedirs("ta_result2/gifts/" + c)